In [1]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="mistral")


In [4]:
llm.invoke("what model are you")

" I am a model of the Mistral AI's family of models, specifically trained as a conversational agent."

In [177]:
from langchain_ollama import OllamaLLM
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, END

In [178]:
llm = OllamaLLM(model="mistral")

In [179]:
def get_weather(location):
    return f"The weather in {location} is sunny and 25\u00b0C."

In [180]:
def get_time(location):
    return f"The time in {location} is 3 PM."

In [181]:
from langchain.tools import Tool

In [191]:
tools = [
    Tool(name="get_weather",func=get_weather,description="Get current weather for a location."),
    Tool(name="get_time",func=get_time,description="Get current time for a location.")
]

In [183]:
tools = {
    "get_weather":get_weather,
    "get_time":get_time
}

In [184]:
def define_state():
    return {"messages":[]}

In [185]:
def agent_node(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    state["messages"].append(AIMessage(content=response))
    return state

In [199]:
system_prompt = """
You are a helpful AI assistant with access to two tools. Use a tool ONLY if the user explicitly asks about the weather or time in a specific location. For all other questions, respond normally and conversationally like a friendly assistant.

Available tools:
- get_weather: Use this only when the user asks about the weather in a location.
- get_time: Use this only when the user asks about the current time in a location.

Instructions (follow strictly):
1. If the user asks about the weather or time in a specific location, DO NOT respond with anything except a tool call.
2. When using a tool, your response MUST be exactly this format (on a single line, nothing else):
   USE TOOL: tool_name, {"location": "value"}

3. Do NOT:
   - Respond with extra words, line breaks, or explanations when using tools.
   - Predict or explain tool results.
   - Use a tool unless the request is clearly about weather or time.

4. For all other topics, respond normally like a helpful and friendly assistant.

✅ Examples:

User: What’s the weather in New York?
Assistant: USE TOOL: get_weather, {"location": "New York"}

User: What time is it in Paris?
Assistant: USE TOOL: get_time, {"location": "Paris"}

User: Tell me a joke.
Assistant: Why don’t skeletons fight each other? They don’t have the guts.

User: Who are you?
Assistant: I'm your AI assistant. I can help you with questions and fetch weather or time info if needed!

Important:
- If a tool is needed, respond ONLY with a single-line tool call.
- If no tool is needed, act like a regular assistant and answer normally.
"""


In [200]:
msg = agent_node({"messages" : [SystemMessage(content=system_prompt),HumanMessage(content="what is time in paris")]})

In [201]:
import os,json

In [202]:
def tool_node(state):
    last_msg = state["messages"][-1].content

    import re
    match = re.search(r'USE TOOL:\s*(\w+),\s*(\{.*\})', last_msg)
    if match:
        tool_name = match.group(1)
        args = json.loads(match.group(2))
        tool = next((t for t in tools if t.name == tool_name), None)
        if tool:
            result = tool.func(**args)  # safely unpack args
            tool_msg = ToolMessage(
                content=result,
                name=tool_name,
                tool_call_id=f"{tool_name}-1"
            )
            state["messages"].append(tool_msg)
    return state
    


In [203]:
new_msg = tool_node(msg)

In [204]:
tool_content = new_msg["messages"][-1].content

In [205]:
new_msg['messages'].append(AIMessage(content=tool_content))

In [209]:
new_msg['messages'].append(HumanMessage(content="Tell me few things about india"))

In [210]:
new_msg

{'messages': [SystemMessage(content='\nYou are a helpful AI assistant with access to two tools. Use a tool ONLY if the user explicitly asks about the weather or time in a specific location. For all other questions, respond normally and conversationally like a friendly assistant.\n\nAvailable tools:\n- get_weather: Use this only when the user asks about the weather in a location.\n- get_time: Use this only when the user asks about the current time in a location.\n\nInstructions (follow strictly):\n1. If the user asks about the weather or time in a specific location, DO NOT respond with anything except a tool call.\n2. When using a tool, your response MUST be exactly this format (on a single line, nothing else):\n   USE TOOL: tool_name, {"location": "value"}\n\n3. Do NOT:\n   - Respond with extra words, line breaks, or explanations when using tools.\n   - Predict or explain tool results.\n   - Use a tool unless the request is clearly about weather or time.\n\n4. For all other topics, res

In [211]:
agent_node(new_msg)

{'messages': [SystemMessage(content='\nYou are a helpful AI assistant with access to two tools. Use a tool ONLY if the user explicitly asks about the weather or time in a specific location. For all other questions, respond normally and conversationally like a friendly assistant.\n\nAvailable tools:\n- get_weather: Use this only when the user asks about the weather in a location.\n- get_time: Use this only when the user asks about the current time in a location.\n\nInstructions (follow strictly):\n1. If the user asks about the weather or time in a specific location, DO NOT respond with anything except a tool call.\n2. When using a tool, your response MUST be exactly this format (on a single line, nothing else):\n   USE TOOL: tool_name, {"location": "value"}\n\n3. Do NOT:\n   - Respond with extra words, line breaks, or explanations when using tools.\n   - Predict or explain tool results.\n   - Use a tool unless the request is clearly about weather or time.\n\n4. For all other topics, res

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

# Step 1: Initialize the workflow graph
workflow = StateGraph()

# Step 2: Register nodes (these are your functions: llm_node, tool_node, final_response_node)
workflow.add_node("llm", llm_node)           # Node to invoke the LLM
workflow.add_node("tool", tool_node)         # Node to process tool usage
workflow.add_node("final", final_response_node)  # Node to return final response

# Step 3: Define the edges (control flow between nodes)
workflow.set_entry_point("llm")              # Entry point is the LLM node
workflow.add_edge("llm", "tool")             # After LLM, go to tool execution
workflow.add_edge("tool", "final")           # After tool, go to final formatting
workflow.add_edge("final", END)              # Final node ends the graph

# Step 4: Compile the graph
graph = workflow.compile()
